# 重塑与轴向旋转

重新排列表格式数据的操作称为重塑（reshape），也称为轴向旋转(pivot)或透视。在 Pandas 中，重塑与轴向旋转的操作函数有：
- `stack()`：将数据的列“旋转”为行；
- `unstack()`：将数据的行“旋转”为列；
- `pivot()`: 透视操作

本节介绍这几个重塑操作函数的使用。

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity='all'

## 使用多层索引进行重塑

`df.stack()`函数会将列“旋转”为行，产生一个新对象，其使用语法为：
```python
df.stack(level=-1, dropna=True)
```
主要参数
- `level=-1`，指定旋转的索引层级，可为整数、字符串或列表。缺省为最里层行标签。
- `dropna=True`，删除有缺失值的行。

例如下面创建的`DataFrame`对象是多家门店的不同餐时的收入：

In [2]:
df = pd.DataFrame([[872.0, 2489.5, 27028.9], [160.0, 3029.4, 15293.0], [302.0, 5649.2, 20392.0]],
                  index=pd.Index(['Xuhui', 'Sheshan', 'Jingan'], name='门店'),
                  columns = pd.Index(['早点', '午餐', '夜市'], name='餐时'))
df

餐时,早点,午餐,夜市
门店,,,
Xuhui,872.0,2489.5,27028.9
Sheshan,160.0,3029.4,15293.0
Jingan,302.0,5649.2,20392.0


使用`stack()`方法会将列旋转到行，产生一个`Series`对象：

In [3]:
result = df.stack()
result

门店       餐时
Xuhui    早点      872.0
         午餐     2489.5
         夜市    27028.9
Sheshan  早点      160.0
         午餐     3029.4
         夜市    15293.0
Jingan   早点      302.0
         午餐     5649.2
         夜市    20392.0
dtype: float64

反之，使用`unstack()`方法，可以将数据的行旋转为列，其使用语法为：
```python
s.unstack(level=-1, fill_value=None)
```
主要参数
- `level=-1`，指定旋转的索引层级。
- `fill_value=None`，填充缺失值。

例如，上面的结果运行`unstack()`，会得到新的`DataFrame`对象：

In [4]:
result.unstack()

餐时,早点,午餐,夜市
门店,,,
Xuhui,872.0,2489.5,27028.9
Sheshan,160.0,3029.4,15293.0
Jingan,302.0,5649.2,20392.0


缺省是把索引最里层进行旋转，可以指定使用标签索引“门店”：

In [5]:
# 指定标签（字符串或整数）
result.unstack('门店')

门店,Xuhui,Sheshan,Jingan
餐时,,,
早点,872.0,160.0,302.0
午餐,2489.5,3029.4,5649.2
夜市,27028.9,15293.0,20392.0


在使用`stack()`把列旋转为行时，如果行中有缺失值，会删除掉改行：

In [6]:
df2 = pd.DataFrame([[872.0, 2489.5, 27028.9], [np.NAN, 3029.4, 15293.0], [302.0, 5649.2, 20392.0]],
                   index=pd.Index(['Xuhui', 'Sheshan', 'Jingan'], name='门店'),
                   columns = pd.Index(['早点', '午餐', '夜市'], name='餐时'))
df2

餐时,早点,午餐,夜市
门店,,,
Xuhui,872.0,2489.5,27028.9
Sheshan,NaN,3029.4,15293.0
Jingan,302.0,5649.2,20392.0


In [7]:
result2 = df2.stack()
result2

门店       餐时
Xuhui    早点      872.0
         午餐     2489.5
         夜市    27028.9
Sheshan  午餐     3029.4
         夜市    15293.0
Jingan   早点      302.0
         午餐     5649.2
         夜市    20392.0
dtype: float64

通过传入`dropna=False`可以避免此种情况。同样在使用`unstack()`把行旋转为列时，㘝层级中所有值不齐全时，拆分可能会引入缺失值，使用`fill_value`参数可以指定值来填充缺失值。例如下面结果:

In [8]:
result2.unstack()
# 指定值填充
# result2.unstack(fill_value=0)

餐时,早点,午餐,夜市
门店,,,
Xuhui,872.0,2489.5,27028.9
Sheshan,NaN,3029.4,15293.0
Jingan,302.0,5649.2,20392.0


## “长”格式与“宽”格式的转换

收集数据时会在不同地点与时间进行观测采集，表中的每一行表示某个地点与某个时间的单个观测值。这种具有两个或多个键的观测数据常称为长格式数据。在数据库中，不断地采集数据，数据集也会不断增长。使用`pivot()`可以把长格式转换为宽格式数据，其使用语法为：
```python
df.pivot(index=None, columns=None, values=None)
```
主要参数
- `index=None`，字符串或对象，指定新对象行索引。缺省使用原对象索引。
- `columns=None`，字符串或对象，指定新对象列索引。缺省使用原对象索引。
- `valuesNone`，字符串，对象或列表，指定新对象数据。缺省使用原对象剩余列。

例如，下面创建一个`DataFrame`对象，表示不同门店，不同用餐时段的顾客人数与流水数。

In [9]:
df = pd.DataFrame({'门店': ['Xuhui', 'Xuhui', 'Xuhui', 'Sheshan', 'Sheshan', 'Sheshan', 'Jingan', 'Jingan', 'Jingan'],
                   '餐时': ['早点', '午餐', '夜市','早点', '午餐', '夜市', '早点', '午餐', '夜市'],            
                   '顾客': [100, 120, 300, 30, 200, 300, 40, 250, 200],
                   '流水': [872.0, 2489.5, 27028.9, 160.0, 3029.4, 15293.0, 302.0, 5649.2, 20392.0]})
df

,门店,餐时,顾客,流水
0,Xuhui,早点,100,872.0
1,Xuhui,午餐,120,2489.5
2,Xuhui,夜市,300,27028.9
3,Sheshan,早点,30,160.0
4,Sheshan,午餐,200,3029.4
5,Sheshan,夜市,300,15293.0
6,Jingan,早点,40,302.0
7,Jingan,午餐,250,5649.2
8,Jingan,夜市,200,20392.0


使用`pivot()`方法，指定列“门店”为新对象行索引，列“餐时”为列索引，其它剩余列为新对象数据，最后转换宽格式数据为：

In [10]:
df.pivot('门店', '餐时')

顾客                流水                
餐时        午餐   夜市   早点      午餐       夜市     早点
门店                                            
Jingan   250  200   40  5649.2  20392.0  302.0
Sheshan  200  300   30  3029.4  15293.0  160.0
Xuhui    120  300  100  2489.5  27028.9  872.0

生成的宽格式数据的列索引为层级索引。也可以显式地指定新对象的数据列：

In [11]:
df.pivot(index='门店',columns= '餐时', values='顾客')

餐时,午餐,夜市,早点
门店,,,
Jingan,250,200,40
Sheshan,200,300,30
Xuhui,120,300,100


Pandas 提供函数`pd.melt()`来把宽格式透视为长格式，即将多列合并成一列，生成新的`DataFrame`对象，其使用语法为：
```python
df.melt(id_vars=None, value_vars=None, var_name=None, value_name='value', col_level=None)
```
主要参数
- `id_vars`，元组、列表或类数组对象。指定分组列。
- `value_vars`，元组、列表或类数组对象。指定数据列。

例如下面创建一个宽格式数据：

In [12]:
data = pd.DataFrame({'门店': ['Xuhui', 'Xuhui', 'Xuhui', 'Sheshan', 'Sheshan', 'Sheshan', 'Jingan', 'Jingan', 'Jingan'],
                     '餐时': ['早点', '午餐', '夜市','早点', '午餐', '夜市', '早点', '午餐', '夜市'],            
                     '流水': [872.0, 2489.5, 27028.9, 160.0, 3029.4, 15293.0, 302.0, 5649.2, 20392.0]})
df = data.pivot(index='门店',columns= '餐时', values='流水').reset_index()
df

餐时,门店,午餐,夜市,早点
0,Jingan,5649.2,20392.0,302.0
1,Sheshan,3029.4,15293.0,160.0
2,Xuhui,2489.5,27028.9,872.0


使用`melt()`方法转换为长格式数据，例如指定分组列为“门店”，指定数据列为“早点，午餐”：

In [13]:
df.melt(id_vars=['门店'], value_vars=['早点', '午餐'])

,门店,餐时,value
0,Jingan,早点,302.0
1,Sheshan,早点,160.0
2,Xuhui,早点,872.0
3,Jingan,午餐,5649.2
4,Sheshan,午餐,3029.4
5,Xuhui,午餐,2489.5


如果不指定数据列，会使用剩余数据列：

In [14]:
df.melt(id_vars=['门店'])

,门店,餐时,value
0,Jingan,午餐,5649.2
1,Sheshan,午餐,3029.4
2,Xuhui,午餐,2489.5
3,Jingan,夜市,20392.0
4,Sheshan,夜市,15293.0
5,Xuhui,夜市,27028.9
6,Jingan,早点,302.0
7,Sheshan,早点,160.0
8,Xuhui,早点,872.0
